In [ ]:
import cctx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ----------------------------------------------------------
# 1. 데이터 수집 및 가공
# ----------------------------------------------------------
binance = ccxt.binance()
print("데이터 수집 및 통계 계산 중... (약 5초 소요)")

def fetch_data(symbol):
    # 넉넉하게 200일치 데이터를 가져옵니다.
    ohlcv = binance.fetch_ohlcv(symbol, '1d', limit=200)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df[['timestamp', 'close']]

btc = fetch_data('BTC/USDT')
eth = fetch_data('ETH/USDT')
df = pd.merge(btc, eth, on='timestamp', suffixes=('_BTC', '_ETH'))

# 통계 지표 산출
df['ratio'] = df['close_BTC'] / df['close_ETH']
df['mean'] = df['ratio'].rolling(window=20).mean()
df['std'] = df['ratio'].rolling(window=20).std()
df['z_score'] = (df['ratio'] - df['mean']) / df['std']

# ----------------------------------------------------------------
# 2. 백테스팅 (Mark-to-Market: 평가담보금 반영)
# ----------------------------------------------------------------
balance = 1000.0 # 확정 잔고 (Cash)
equity = 1000.0 # 평가 잔고 (Cash + Unrealized PnL)
fee_rate = 0.001 # 수수료 0.1%

position = None # None, 'short_ratio', 'long_ratio'
entry_price_btc = 0
entry_price_eth = 0
entry_balance = 0 # 진입 시점의 확정 잔고

history = []

for index, row in df.iterrows():
    # 데이터가 없으면 패스 (초기 20일)
    if np.isnan(row['z_score']):
        continue

    z = row['z_score']
    price_btc = row['close_BTC']
    price_eth = row['close_ETH']

    # [평가 손익 계산] 포지션이 있을 때 매일매일의 가치를 추적
    unrealized_pnl = 0
    if position is not None:
        btc_ret = (price_btc - entry_price_btc) / entry_price_btc
        